In [230]:
import pandas as pd
import requests
import time

In [282]:
gps_inp = "../../samples/data/atuanhm_2705_gps_processed.csv"
vehicle_inp = "../../samples/data/atuanhm_2705_report_vehicle_processed.csv"

id = "sample car 4"
delay=0.05

In [283]:
tracking_gps_url = "http://localhost:1234/trackMany"
tracking_vehicle_url = "http://localhost:1234/track"

In [284]:
import json

def to_json(row, id=None):
    obj = {}
    obj['time'] = row['positionTime'] * 1000
    obj['lat'] = row['lat']
    obj['lng'] = row['lng']
    obj['azimuth'] = row['heading']
    if 'speed' in row:
        obj['velocity'] = row['speed']
    if 'height' in row:
        obj['height'] = row['height']
    if id is not None:
        obj["id"] = id
        
    return json.dumps(obj)

In [285]:
def merge_gps(series, id=None):
    res = []
    for u in range(series.shape[0]):
        res.append(json.loads(series.iloc[u]['json']))
        
    obj = {"samples": res}
    if id is not None:
        obj["id"] = id
    
    return json.dumps(obj)

In [286]:
gps_df = pd.read_csv(gps_inp)
gps_df['json'] = gps_df.apply(lambda u: to_json(u), axis=1)

In [287]:
gps_df.head()

,ts,index_record,heading,lat,lng,positionTime,h_positionTime,h_receiveTime,json
0,1622119844873,0,26,21.03023,105.78133,1622119835,2021-05-27 19:50:35,2021-05-27 19:50:44,"{""lat"": 21.03023, ""lng"": 105.78133000000001, ""..."
1,1622119844873,1,26,21.03023,105.78133,1622119836,2021-05-27 19:50:36,2021-05-27 19:50:44,"{""lat"": 21.03023, ""lng"": 105.78133000000001, ""..."
2,1622119844873,2,26,21.03023,105.78134,1622119837,2021-05-27 19:50:37,2021-05-27 19:50:44,"{""lat"": 21.03023, ""lng"": 105.78133999999999, ""..."
3,1622119844873,3,26,21.03024,105.78134,1622119838,2021-05-27 19:50:38,2021-05-27 19:50:44,"{""lat"": 21.03024, ""lng"": 105.78133999999999, ""..."
4,1622119844873,4,26,21.03023,105.78134,1622119839,2021-05-27 19:50:39,2021-05-27 19:50:44,"{""lat"": 21.03023, ""lng"": 105.78133999999999, ""..."


In [288]:
vehicle_df = pd.read_csv(vehicle_inp)
vehicle_df['json'] = vehicle_df.apply(lambda r: to_json(r, id), axis=1)
vehicle_df['type'] = 'vehicle'

### Merge

In [289]:
gps_merge = pd.DataFrame()
gps_merge = gps_df.groupby(by='ts').apply(lambda row: merge_gps(row, id))

gps_merge = gps_merge.reset_index()
gps_merge.columns = ['ts', 'json']
gps_merge['type'] = 'gps'

In [290]:
gps_merge['json'][0]

'{"id": "sample car 4", "samples": [{"lat": 21.03023, "lng": 105.78133000000001, "time": 1622119835000, "azimuth": 26}, {"lat": 21.03023, "lng": 105.78133000000001, "time": 1622119836000, "azimuth": 26}, {"lat": 21.03023, "lng": 105.78133999999999, "time": 1622119837000, "azimuth": 26}, {"lat": 21.03024, "lng": 105.78133999999999, "time": 1622119838000, "azimuth": 26}, {"lat": 21.03023, "lng": 105.78133999999999, "time": 1622119839000, "azimuth": 26}, {"lat": 21.03023, "lng": 105.78133999999999, "time": 1622119840000, "azimuth": 26}]}'

In [291]:
merge_all = pd.concat([gps_merge, vehicle_df[['ts', 'json', 'type']]], ignore_index=True)

In [292]:
merge_all.sort_values(by='ts', inplace=True)

In [293]:
for i in range(merge_all.shape[0]):
    row = merge_all.loc[i]
#     print row
    push = True
    if row['type'] == 'gps':
        url = tracking_gps_url
        push = False
    else:
        push = True
        url = tracking_vehicle_url
    
    if push:
        x = requests.post(url, data=row['json'])
        print(row['json'])
        print(x.text)
        time.sleep(delay)

{"time": 1622119846000, "height": 24, "azimuth": 18, "lat": 21.03021, "velocity": 8, "lng": 105.7813, "id": "sample car 4"}
"SUCCESS"
{"time": 1622119854000, "height": 29, "azimuth": 59, "lat": 21.03027, "velocity": 18, "lng": 105.78131, "id": "sample car 4"}
"SUCCESS"
{"time": 1622119862000, "height": 12, "azimuth": 97, "lat": 21.03061, "velocity": 21, "lng": 105.78126999999999, "id": "sample car 4"}
"SUCCESS"
{"time": 1622119870000, "height": -10, "azimuth": 4, "lat": 21.03105, "velocity": 21, "lng": 105.7813, "id": "sample car 4"}
"SUCCESS"
{"time": 1622119878000, "height": 15, "azimuth": 38, "lat": 21.03126, "velocity": 25, "lng": 105.78138, "id": "sample car 4"}
"SUCCESS"
{"time": 1622119887000, "height": 16, "azimuth": 20, "lat": 21.03133, "velocity": 29, "lng": 105.78097, "id": "sample car 4"}
"SUCCESS"
{"time": 1622119895000, "height": 8, "azimuth": 15, "lat": 21.03131, "velocity": 29, "lng": 105.78048999999999, "id": "sample car 4"}
"SUCCESS"
{"time": 1622119904000, "height": 

{"time": 1622120441000, "height": 1, "azimuth": 38, "lat": 21.00841, "velocity": 38, "lng": 105.7545, "id": "sample car 4"}
"SUCCESS"
{"time": 1622120449000, "height": 0, "azimuth": 37, "lat": 21.00863, "velocity": 29, "lng": 105.75395, "id": "sample car 4"}
"SUCCESS"
{"time": 1622120458000, "height": 1, "azimuth": 28, "lat": 21.00889, "velocity": 45, "lng": 105.75316000000001, "id": "sample car 4"}
"SUCCESS"
{"time": 1622120466000, "height": 1, "azimuth": 27, "lat": 21.00909, "velocity": 49, "lng": 105.75218999999998, "id": "sample car 4"}
"SUCCESS"
{"time": 1622120474000, "height": 3, "azimuth": 27, "lat": 21.00931, "velocity": 52, "lng": 105.75117, "id": "sample car 4"}
"SUCCESS"
{"time": 1622120482000, "height": 3, "azimuth": 25, "lat": 21.009520000000002, "velocity": 58, "lng": 105.75001999999999, "id": "sample car 4"}
"SUCCESS"
{"time": 1622120491000, "height": 2, "azimuth": 21, "lat": 21.0097, "velocity": 63, "lng": 105.74863, "id": "sample car 4"}
"SUCCESS"
{"time": 16221204990

{"time": 1622120960000, "height": 6, "azimuth": 26, "lat": 20.998720000000002, "velocity": 24, "lng": 105.72124, "id": "sample car 4"}
"SUCCESS"
{"time": 1622120968000, "height": 7, "azimuth": 15, "lat": 20.99875, "velocity": 21, "lng": 105.72083, "id": "sample car 4"}
"SUCCESS"
{"time": 1622120977000, "height": 6, "azimuth": 8, "lat": 20.99871, "velocity": 17, "lng": 105.72045, "id": "sample car 4"}
"SUCCESS"
{"time": 1622120985000, "height": 7, "azimuth": 39, "lat": 20.9987, "velocity": 17, "lng": 105.72012, "id": "sample car 4"}
"SUCCESS"
{"time": 1622120993000, "height": 8, "azimuth": 43, "lat": 20.998929999999998, "velocity": 17, "lng": 105.71985, "id": "sample car 4"}
"SUCCESS"
{"time": 1622121001000, "height": 9, "azimuth": 33, "lat": 20.999000000000002, "velocity": 12, "lng": 105.71963999999998, "id": "sample car 4"}
"SUCCESS"
{"time": 1622121010000, "height": 8, "azimuth": 50, "lat": 20.99902, "velocity": 2, "lng": 105.71961999999999, "id": "sample car 4"}
"SUCCESS"
{"time": 1